# This notebook is part 1 of using autogen with local ai models to query an arbitrary database.
In this notebook I will simply connect ChatGPT to a local database and query it.  I will not be using autogen or local AI models
I'm using this reference https://github.com/disler/multi-agent-postgres-data-analytics/tree/v1-prompt-engineering-an-entire-codebase

## Install ODBC Driver for MSSQL
If connecting to a SQL database from wsl ubuntu (like this notebook), you will need to install the following packages:
```bash
sudo apt install unixodbc
```

Then follow instructions here
https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver15&tabs=alpine18-install%2Cubuntu17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline

basically run this bash script for version 17 (other versions available see link)
```
if ! [[ "16.04 18.04 20.04 22.04" == *"$(lsb_release -rs)"* ]];
then
    echo "Ubuntu $(lsb_release -rs) is not currently supported.";
    exit;
fi

curl https://packages.microsoft.com/keys/microsoft.asc | sudo tee /etc/apt/trusted.gpg.d/microsoft.asc

curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list

sudo apt-get update
sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
# optional: for bcp and sqlcmd
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools
echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bashrc
source ~/.bashrc
# optional: for unixODBC development headers
sudo apt-get install -y unixodbc-dev
```

In [2]:
%pip install pyodbc
%pip install python-dotenv
%pip install --upgrade --force-reinstall openai==0.28.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached openai-0.28.1-py3-none-any.whl.metadata (11 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached aiohttp-3.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl.metadata (2.2 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached multidict-6.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached yarl-1.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
  Using cached frozenlist-1.4.0-cp310-cp310-manylinux_2_5_x86_64.many

## Fetch the required environment variables

In [10]:
import os
import dotenv

dotenv.load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

## Create code to handle connecting to the database
The settings for the database should obviously be in a config file, but for now I'm just going to hard code them in.

In [15]:
# connect to MSSQL database
import pyodbc

SERVER = '127.0.0.1'
DATABASE = 'TestDb'
USERNAME = 'sa'
PASSWORD = 'Password123'

connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

conn = pyodbc.connect(connectionString)

# conn.close()

# Get all table definitions
The table information will be passed to the AI prompt so it can create queries.

In [4]:
# Get table names
get_all_tables_stmt = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES;"
cursor = conn.cursor()
cursor.execute(get_all_tables_stmt)
table_names = [row[0] for row in cursor.fetchall()]

# Get all table definitions and format them into CREATE TABLE statements
get_def_stmt = f"""
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = ?
"""

definitions = []
for table_name in table_names:
    cursor.execute(get_def_stmt, (table_name,))
    rows = cursor.fetchall()

    create_table_stmt = f"CREATE TABLE {table_name} (\n"
    for row in rows:
        create_table_stmt += f"{row[0]} {row[1]},\n"
    create_table_stmt = create_table_stmt.rstrip(",\n") + "\n);"
    
    definitions.append(create_table_stmt)

table_definitions = "\n\n".join(definitions)

## Setup the AI model

In [5]:
import openai

openai.api_key = os.environ.get("OPENAI_API_KEY")

## Build the prompt

In [ ]:
# This is the user input
user_question = "How many users are there in the database?"

# The rest of this prompt building is back end logic
## Build the request
prompt = f"Fulfill this database query: {user_question}. "
POSTGRES_TABLE_DEFINITIONS_CAP_REF = "TABLE_DEFINITIONS"
prompt_suffix = f"Use these {POSTGRES_TABLE_DEFINITIONS_CAP_REF} to satisfy the database query."

prompt = f"""{prompt} {prompt_suffix}\n\n{POSTGRES_TABLE_DEFINITIONS_CAP_REF}\n\n{table_definitions}"""

## Append the response format
SQL_DELIMITER = "---------"
RESPONSE_FORMAT_CAP_REF = "RESPONSE_FORMAT"
prompt_suffix = f"\n\nRespond in this format {RESPONSE_FORMAT_CAP_REF}. Replace the text between <> with it's request. I need to be able to easily parse the sql query from your response."
cap_ref_content = f"""<explanation of the sql query>
{SQL_DELIMITER}
<sql query exclusively as raw text>"""
prompt = f"""{prompt} {prompt_suffix}\n\n{RESPONSE_FORMAT_CAP_REF}\n\n{cap_ref_content}"""

print("\n\n-------- PROMPT --------")
print(prompt)

## Send the prompt to the AI model
NOTE: I had to pin the openai version to 0.28 becuase the 1.0 version has breaking changes.  To keep this example short I did not want to go through the migration guide.
https://github.com/openai/openai-python/discussions/742

In [13]:
model = "gpt-4"
openai.api_key = OPENAI_API_KEY

response = openai.ChatCompletion.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
)

print("\n\n-------- RESPONSE --------")
print(response)



-------- RESPONSE --------
{
  "id": "chatcmpl-8SsLY8s8a2HiTncm1vdlNaRrjMyhG",
  "object": "chat.completion",
  "created": 1701892784,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The SQL query counts the number of unique user IDs in the tblUsers table. We use the COUNT function with the DISTINCT keyword to ensure that each individual user is only counted once, even if they may appear multiple times in the table.\n---------\nSELECT COUNT(DISTINCT id) FROM tblUsers;"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 449,
    "completion_tokens": 59,
    "total_tokens": 508
  },
  "system_fingerprint": null
}


## Format the response and extract the SQL query

In [14]:
# This was taken directly from the example.  I need to review it later.
def safe_get(data, dot_chained_keys):
    """
    {'a': {'b': [{'c': 1}]}}
    safe_get(data, 'a.b.0.c') -> 1
    """
    keys = dot_chained_keys.split(".")
    for key in keys:
        try:
            if isinstance(data, list):
                data = data[int(key)]
            else:
                data = data[key]
        except (KeyError, TypeError, IndexError):
            return None
    return data

prompt_response = safe_get(response, "choices.0.message.content")

sql_query = prompt_response.split(SQL_DELIMITER)[1].strip()

print(f"\n\n-------- PARSED SQL QUERY --------")
print(sql_query)



-------- PARSED SQL QUERY --------
SELECT COUNT(DISTINCT id) FROM tblUsers;


## Execute the query and print the results
Also make sure to close the database connection here!

In [16]:
cursor = conn.cursor()
cursor.execute(sql_query)
rows = cursor.fetchall()
print(f"\n\n-------- SQL QUERY RESULTS --------")
print(rows)
conn.close()



-------- SQL QUERY RESULTS --------
[(150,)]
